In [1]:
import pandas as pd
import numpy as np
import random

In [2]:
class player:
    def __init__(self):
        self.final_result = ["B", "I", "N", "G", "O"]
        self.full_list = [num for num in range(1, 26)]
        random.shuffle(self.full_list)
        self.grid = np.reshape(self.full_list, newshape=(5,5))
        self.bool_grid = np.zeros((5,5))
        self.bool_grid = self.bool_grid == 1
        self.message = ""
        self.dup_grid = []
        for i in range(5):
            row = []
            for j in range(5):
                row.append(str(self.grid[i,j]).zfill(2))
            self.dup_grid.append(row)
        
    def update_bool_grid(self, user_input):
        i, j = np.transpose((self.grid == user_input).nonzero())[0]
        self.bool_grid[i, j] = True
        self.dup_grid[i][j] = "**"
    
    def player_turn(self, user_input):
        self.update_bool_grid(user_input)
        counter = self.bingo_counter()
        message = ""
        if counter > 0:
            self.message = "".join(self.final_result[:counter])
    
    def bingo_counter(self):
        counter = 0
        on_diag = []
        off_diag = []
        for i in range(5):
            if sum(self.bool_grid[i,:]) == 5:
                counter += 1
            if sum(self.bool_grid[:, i]) == 5:
                counter += 1
            on_diag.append(self.bool_grid[i,i])
            off_diag.append(self.bool_grid[i, 4-i])
        if sum(on_diag) == 5:
            counter += 1
        if sum(off_diag) == 5:
            counter += 1
        return counter
    
    def print_grid(self):
        for i, row in enumerate(self.bool_grid):
            for j, column in enumerate(row):
                if column:
                    print("|**", end="")
                else:
                    print(f"|{self.dup_grid[i][j]}", end="")
            print("|")

In [3]:
class user(player):
    def next_choice(self, user_input_list, level=""):
        user_input = input("Enter a number: ")
        try:
            user_input = int(user_input)
        except Exception:
            print("Not integer. Please enter an integer.")
            self.next_choice()
        if user_input in user_input_list:
            user_input_list = list(set(user_input_list) - set([user_input]))
            return user_input, user_input_list
        else:
            print("Not between 1 and 25. Enter again")
            self.next_choice()
            
class computer(player):
    def random_choice(self, user_input_list):
        user_input = random.choices(user_input_list, k=1)[0]
        user_input_list = list(set(user_input_list) - set([user_input]))
        return user_input, user_input_list
    
    def row_opt_choice(self, user_input_list):
        grid_ = np.array(self.dup_grid)
        possible_values = np.transpose((grid_ != "**").nonzero())
        unique, counts = np.unique(possible_values.transpose()[0], return_counts=True)
        row =  [unique[i] for i,num in enumerate(counts) if num == 1]
        if len(row) > 0:
            row = row[0]
            i,j = [value for value in possible_values if value[0]==row][0]
            user_input = int(grid_[i,j])
            user_input_list = list(set(user_input_list) - set([user_input]))
            return user_input, user_input_list
        else:
            return 26, user_input_list
    
    def column_opt_choice(self, user_input_list):
        grid_ = np.array(self.dup_grid)
        possible_values = np.transpose((grid_ != "**").nonzero())
        unique, counts = np.unique(possible_values.transpose()[1], return_counts=True)
        column =  [unique[i] for i,num in enumerate(counts) if num == 1]
        if len(column) > 0:
            column = column[0]
            i,j = [value for value in possible_values if value[1]==column][0]
            user_input = int(grid_[i,j])
            user_input_list = list(set(user_input_list) - set([user_input]))
            return user_input, user_input_list
        else:
            return 26, user_input_list
    
    def next_choice(self, user_input_list, level="easy"):
        if level == "easy":
            user_input, user_input_list = self.random_choice(user_input_list)
        if level == "medium":
            user_input, user_input_list = self.row_opt_choice(user_input_list)
            if user_input == 26:
                user_input, user_input_list = self.column_opt_choice(user_input_list)
            if user_input == 26:
                user_input, user_input_list = self.random_choice(user_input_list)
        return user_input, user_input_list

In [4]:
user_input_list = [num for num in range(1, 26)]

In [5]:
A = computer()
B = user()

In [6]:
first_player = "A"
user = "B"

In [7]:
if first_player == "A":
    turns_ = ["A", "B"]
    players = [A, B]
else:
    turns_ = ["B", "A"]
    players = [B, A]

In [8]:
if first_player == user:
    print(players[0].grid)
else:
    print(players[1].grid)

[[22 16 13 24  7]
 [ 6 18 10  3 20]
 [ 9 11  8 23  2]
 [25 15 19  5  1]
 [14 17 21 12  4]]


In [ ]:
turn_number = 0
flag = 0
while flag==0:
    player_ = turn_number % 2
    turn_number += 1
    player_name = turns_[player_]
    if player_ == 0:
        user_choice, user_input_list = players[0].next_choice(user_input_list, level="medium")
    else:
        user_choice, user_input_list = players[1].next_choice(user_input_list, level="medium")
    players[0].player_turn(user_choice)
    players[1].player_turn(user_choice)
    if (first_player == user):
        print("Your Turn")
        players[0].print_grid()
        print()
        print(f"Your Message: {players[0].message}")
        print(f"Computer's Message: {players[1].message}")
    else:
        print("Your Turn")
        players[1].print_grid()
        print()
        print(f"Your Message: {players[1].message}")
        print(f"Computer's Message: {players[0].message}")
    
    if players[player_].message == "BINGO":
        if player_name == user:
            print("You Win")
        else:
            print("Computer Wins")
        flag = 1

Your Turn
|22|16|13|24|07|
|06|18|10|03|20|
|09|11|08|23|02|
|25|15|19|05|**|
|14|17|21|12|04|

Your Message: 
Computer's Message: 


Enter a number:  22


Your Turn
|**|16|13|24|07|
|06|18|10|03|20|
|09|11|08|23|02|
|25|15|19|05|**|
|14|17|21|12|04|

Your Message: 
Computer's Message: 
Your Turn
|**|16|13|24|07|
|06|18|10|03|20|
|09|11|08|23|02|
|25|15|19|05|**|
|14|17|21|**|04|

Your Message: 
Computer's Message: 


Enter a number:  07


Your Turn
|**|16|13|24|**|
|06|18|10|03|20|
|09|11|08|23|02|
|25|15|19|05|**|
|14|17|21|**|04|

Your Message: 
Computer's Message: 
Your Turn
|**|16|13|24|**|
|06|18|10|03|20|
|09|11|08|23|02|
|25|**|19|05|**|
|14|17|21|**|04|

Your Message: 
Computer's Message: 


...

Your Turn
|**|16|**|24|**|
|06|18|10|03|20|
|09|11|**|23|**|
|25|**|19|05|**|
|**|17|21|**|**|

Your Message: 
Computer's Message: 
Your Turn
|**|16|**|24|**|
|06|18|10|03|20|
|09|**|**|23|**|
|25|**|19|05|**|
|**|17|21|**|**|

Your Message: 
Computer's Message: 


Enter a number:  03


Your Turn
|**|16|**|24|**|
|06|18|10|**|20|
|09|**|**|23|**|
|25|**|19|05|**|
|**|17|21|**|**|

Your Message: B
Computer's Message: 
Your Turn
|**|16|**|24|**|
|06|18|10|**|**|
|09|**|**|23|**|
|25|**|19|05|**|
|**|17|21|**|**|

Your Message: BI
Computer's Message: B


Enter a number:  18


Your Turn
|**|16|**|24|**|
|06|**|10|**|**|
|09|**|**|23|**|
|25|**|19|05|**|
|**|17|21|**|**|

Your Message: BI
Computer's Message: B
Your Turn
|**|16|**|24|**|
|06|**|10|**|**|
|09|**|**|23|**|
|25|**|19|**|**|
|**|17|21|**|**|

Your Message: BIN
Computer's Message: BI
